## 1.Camera Calibration

In [65]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

chessboard_size = (9,6)

def calibrate_camera(directory,image_files,chessboard_size):
    chessboard_point_num = chessboard_size[0]*chessboard_size[1]
    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points in image place

    # Prepare object points
    objp = np.zeros((chessboard_point_num,3),np.float32)
    objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2)

    for image_file in image_files:
        image = cv2.imread(directory+image_file)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        # Find chessboard corners
        ret, corners = cv2.findChessboardCorners(gray,chessboard_size,None)
        
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    
    if(len(objpoints) == 0 or len(imgpoints) == 0):
        raise Error("Calibration Failed")
    print(gray.shape)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints,gray.shape,None,None)    
    return mtx, dist


In [66]:
import os

calbration_directory = 'camera_cal/'
image_files = os.listdir(calbration_directory)
mtx,dist = calibrate_camera(calbration_directory,image_files,chessboard_size)

(720, 1280)


## 2. Distortion Correction

In [67]:
def undistort(image,mtx,dist):
    image = cv2.undistort(image,mtx,dist,mtx)
    return image

In [79]:
chessboard_dist = cv2.imread('camera_cal/calibration2.jpg')
chessboard_undist = undistort(chessboard_dist,mtx,dist)
#f,((ax1,ax2)) = plt.subplots(1,2,figsize=(12,18))
#ax1.imshow(chessboard_dist)
#ax1.set_title('origin')
#ax2.imshow(chessboard_undist)
#ax2.set_title('undist')

## 3. Color Transform

## 4. Perspective Transform

In [ ]:
src_points = np.float32(
    [[253.685],
     [1047,685],
     [686,450],
     [600,450]])

dst_points = np.float32(
    [[253.685],
     [1047,685],
     [1047,50],
     [253,50]])
M_per = cv2.getPerspectiveTransform(src_points,dst_points)
#cv2.warpPerspective(src,M,size,flags=cv2.INTER_LINEAR)


## 5.Finding Lines

In [ ]:
def find_lines(binary_warped,nwindows = 9):
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:],axis = 0)
    out_img = np.dstack((binary_warped,binary_warped,binary_warped))*255
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return left_fit,right_fit

## 6. Measure Curvature

In [ ]:
ym_per_pix = 30/720
xm_per_pix = 3.7/700

def 